In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

df = pd.read_csv("./new_preprocessed_final.csv")

In [2]:
# Select TF‑IDF features (lexical similarities)
tfidf_feats = [c for c in df.columns if ("cosine" in c.lower() and "bert" not in c.lower()) or "overlap" in c.lower()]

# Additional features (exclude BERT & labels)
additional_feats = ["title_length","sensational_word_count","punctuation_count","punctuation_ratio"]

# Filter only existing columns
additional_feats = [c for c in additional_feats if c in df.columns]

# Final feature set = TF‑IDF + token overlap + extra linguistic features
feature_cols = tfidf_feats + additional_feats

In [3]:
# Label
label_col = None
for c in df.columns:
  if "is_clickbait" in c.lower():
    label_col = c
    break

if not label_col:
  raise ValueError("Label kolom tidak ditemukan pada dataset.")

In [4]:
X = df[feature_cols]
y = df[label_col]

# Train‑test split
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.2, random_state=42, stratify=y
)

# LR pipeline
pipe = Pipeline([
  ("scaler", StandardScaler()),
  ("clf", LogisticRegression(max_iter=300, class_weight="balanced"))
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print("== Features used ==")
print(feature_cols)

print("\n=== Classification Report (LR + TF-IDF + Extras) ===")
print(classification_report(y_test, y_pred))

print("\n=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred))

== Features used ==
['cosine_similarity', 'token_overlap', 'title_length', 'sensational_word_count', 'punctuation_count', 'punctuation_ratio']

=== Classification Report (LR + TF-IDF + Extras) ===
              precision    recall  f1-score   support

           0       0.99      0.92      0.95       514
           1       0.66      0.97      0.78        86

    accuracy                           0.92       600
   macro avg       0.83      0.94      0.87       600
weighted avg       0.95      0.92      0.93       600


=== Confusion Matrix ===
[[471  43]
 [  3  83]]
